In [1]:
import pandas as pd
import patsy
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem
import warnings 
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
from scipy.stats import sem
import math
from sklearn.linear_model import *
train = pd.read_csv("/home/moi/Downloads/crime/train.csv")

In [2]:
#Taking a random sample of 10,000, and then fixing the index column
train=train.sample(10000,random_state=13)
train.index=range(10000)

In [3]:
#Get "hour", "year" and "month"
tmh=pd.DataFrame({'Hour':train['Dates'].apply(lambda x: (str.split(str.split(x,':')[0],' ')[1])).astype(int)})
tmy=pd.DataFrame({'Year':train['Dates'].apply(lambda x: str.split(x, '-')[0]).astype(int)})
tmm=pd.DataFrame({'Month':train['Dates'].apply(lambda x: str.split(str.split(x, '-')[1],'-')[0]).astype(int)})

#Create new dataframe with features: X, Y, Hour, Year and Month
thething=pd.concat([tmh,tmy,tmm, train[['X','Y']]],axis=1)

#mean_score function
def mean_score(scores):
    return ("Mean score: {0:.3f} (+/-{1:.3f})").format(np.mean(scores),sem(scores))

#Check scores via cross-validation
clf=Pipeline([('scaler', StandardScaler()), ('logistic', LogisticRegression(multi_class='ovr'))])
cv=KFold(len(thething), 5, shuffle=True, random_state=33)
scores=cross_val_score(clf,thething,train[['Category']],cv=cv)
print('Scores:', scores)
print(mean_score(scores))

Scores: [ 0.206   0.2235  0.204   0.207   0.2065]
Mean score: 0.209 (+/-0.004)


In [4]:
#Adding X and Y coordinates
thething=pd.concat([tmh,tmy,tmm, train[['X','Y']]],axis=1)
scores=cross_val_score(clf,thething,train[['Category']],cv=cv)
print('Scores:', scores)
print(mean_score(scores))

Scores: [ 0.206   0.2235  0.204   0.207   0.2065]
Mean score: 0.209 (+/-0.004)


In [5]:
#Adding more features using one hot encoding

#PdDistrict
TTT=patsy.dmatrix('PdDistrict', data=train) 

#Day of the week
TTD=patsy.dmatrix('DayOfWeek', data=train) 

#Combining into one dataframe
thething=pd.concat([tmh,tmy,tmm, train[['X','Y']], pd.DataFrame(TTT),  pd.DataFrame(TTD)],axis=1)

#checking scores
scores=cross_val_score(clf,thething,train[['Category']],cv=cv)
print('Scores:', scores)
print(mean_score(scores))

Scores: [ 0.218   0.2325  0.2295  0.234   0.2285]
Mean score: 0.229 (+/-0.003)


In [6]:
#Trying a PCA

#Pre-process the dataframe
from sklearn import preprocessing
scaler=preprocessing.StandardScaler().fit(thething)
X=scaler.transform(thething)

#Fit into PCA
from sklearn.decomposition import PCA
estimator=PCA()
X_pca=estimator.fit_transform(X)  

#checking scores for the first 8 features of the PCA
scores=cross_val_score(clf,pd.DataFrame(X_pca)[[0,1,2,3,4,5,6,7,8]],train[['Category']],cv=cv)
print('Scores:', scores)
print(mean_score(scores))

Scores: [ 0.206   0.1985  0.211   0.2055  0.2035]
Mean score: 0.205 (+/-0.002)
